In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd

sys.path.append('./module')

import openpyxl
import edit_excel as ee

In [2]:
input_dir = Path('../data/')
output_dir = Path('../output/')

In [3]:
filelist = os.listdir(input_dir)

In [4]:
file_name = 'sample.csv'
df = pd.read_csv(input_dir/file_name, engine='python', index_col=0)
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
column_list = df.columns

In [6]:
# ワークブックの作成
wb = openpyxl.Workbook()
f_name = file_name.split('.')[0]+'.xlsx'
wb.save(output_dir/f_name)

# ワークブックの呼び出し
wb = openpyxl.load_workbook(output_dir/f_name)

In [7]:
# 最初のページ
ws = wb['Sheet']
ws.row_dimensions[1].height = 5
ws.column_dimensions['A'].width = 1

ws.title = 'データ概要'
ee.set_title(ws, 'データ概要')
ws.cell(3,2).value = 'ファイル名'
ws.cell(3,3).value = file_name

# カラム情報
header = ['No.', '論理名', '物理名', '解釈', '備考']
for i, c in enumerate(header):
    ws.cell(4, 2+i).value = c
    ee.set_style(ws.cell(4, 2+i), 'style_header')

for i, c in enumerate(df.columns):
    ws.cell(5+i, 2).value = i
    ee.set_style(ws.cell(5+i, 2), 'style_header')
    ws.cell(5+i, 3).value = c
    
ee.adjust_width(ws)

In [8]:
# 次のページ
ws = wb.create_sheet(title='統計量')
ws = ee.set_title(ws, '統計量')

ws.row_dimensions[1].height = 5
ws.column_dimensions['A'].width = 1

In [9]:
tmp1 = pd.DataFrame(
    [df.isnull().sum().values,
     df.isnull().sum().values / len(df) * 100],
    columns = df.columns,
    index = ['欠損数', '欠損率']
)

tmp2 = df.describe(include='all')
tmp2.index = ['行数', 'ﾕﾆｰｸ数', '先頭行', '最頻値', '平均', '標準偏差', '最小', '25%', '中央値', '75%', '最大']

# 統計量記入
describe = pd.concat([tmp1, tmp2])
for c in describe.columns:
    if describe[c].dtype != 'object':
        describe[c] = describe[c].round(3)
ws = ee.put_dataframe(ws, describe, start=(3,2))
        
# 行数、列数
ws.cell(17, 2).value = '行'
ee.set_style(ws.cell(17,2), 'style_header')
ws.cell(17, 3).value = df.shape[0]
ws.cell(17, 4).value = '列'
ee.set_style(ws.cell(17,4), 'style_header')
ws.cell(17, 5).value = df.shape[1]

# 先頭、末尾
ws = ee.put_dataframe(ws, df.head(20), start=(19,2))
ws = ee.put_dataframe(ws, df.tail(20), start=(41,2))

ee.adjust_width(ws)

In [10]:
describe

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
欠損数,0.000,0.000,0,0,177.000,0.000,0.000,0,0.000,687,2
欠損率,0.000,0.000,0,0,19.865,0.000,0.000,0,0.000,77.1044,0.224467
行数,891.000,891.000,891,891,714.000,891.000,891.000,891,891.000,204,889
ﾕﾆｰｸ数,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
先頭行,NaN,NaN,"Hanna, Mr. Mansour",male,NaN,NaN,NaN,CA. 2343,NaN,C23 C25 C27,S
最頻値,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
平均,0.384,2.309,NaN,NaN,29.699,0.523,0.382,NaN,32.204,NaN,NaN
標準偏差,0.487,0.836,NaN,NaN,14.526,1.103,0.806,NaN,49.693,NaN,NaN
最小,0.000,1.000,NaN,NaN,0.420,0.000,0.000,NaN,0.000,NaN,NaN
25%,0.000,2.000,NaN,NaN,20.125,0.000,0.000,NaN,7.910,NaN,NaN


In [11]:
ws.cell(1,1).column

1

In [12]:
wb.save(output_dir/f_name)